In [156]:
import numpy as np
import math as m
import scipy as sp
import matplotlib.pyplot as plt

First we implement a class to represent a normal random variable.

In [104]:
class NormalRV:
    """
    Class for a multivariate normal random variable.
    """

    def __init__(self, mu, Sigma):
        """
        Constructor. Sets:
        - mu (np.array): mean
        - Sigma (np.matrix): covariance matrix
        - S (np.matrix): inverse of Sigma
        - m (np.array): S@mu
        """
        self.mu = mu
        self.Sigma = Sigma
        self.S = np.linalg.inv(Sigma)
        self.m = self.S@self.mu
    
    def pdf(self, x):
        """
        Evaluates the pdf at x. x can be a vector or a matrix; in the latter case, 
        the pdf is evaluated column-wise.

        Args:
        - x (np.array): point(s) at which to evaluate the pdf

        Returns:
        - pdf (np.array): pdf evaluated at x
        """

        mu = self.mu
        Sigma = self.Sigma
        d = len(mu)
        x = np.asarray(x)
        if x.ndim == 1:
            x = x[:, np.newaxis]
        n = x.shape[1]
        pdf = np.zeros(n)
        for i in range(n):
            pdf[i] = 1/np.sqrt((2*np.pi)**d * np.linalg.det(Sigma)) * \
                np.exp(-1/2 * (x[:,i] - mu).T @ np.linalg.inv(Sigma) @ (x[:,i] - mu))
        return pdf

    def sample(self, n):
        """
        Draws n samples from the distribution.

        Args:
        - n (int): number of samples to draw

        Returns:
        - samples (np.array): samples drawn from the distribution
        """

        mu = self.mu
        Sigma = self.Sigma
        return np.random.multivariate_normal(mu, Sigma, n).T
    
    def update_vals(self, m, S):
        """
        Updates the mean and covariance matrix of the distribution.

        Args:
        - m (np.array): new mean
        - S (np.matrix): new covariance matrix

        Sets:
        - mu (np.array): mean
        - Sigma (np.matrix): covariance matrix
        - S (np.matrix): inverse of Sigma
        - m (np.array): S@mu
        """

        self.m = m
        self.S = S
        self.Sigma = np.linalg.inv(S)
        self.mu = self.Sigma@m
        

Note that our update step is:

$$
\theta_{n+1}\leftarrow \theta_n - \alpha \nabla \rho(\theta)
$$

In our case, as both distributions are normal, we can use the fundamental representation of the normal distribution so that its pdf is in the exponential family. This means that:

$$
\nabla \rho(\theta) = \int_{\mathbb{R}^n}-\frac{\pi^2(x)}{q^2_\theta(x)}\nabla\log q_\theta(x) dx
$$

Then, as $\theta = (\mu, \Sigma)$ and using the fundamental representation of the normal distribution we arrive to:

$$
\begin{align*}
\frac{\partial \log}{\partial m}(x) &= x-S^{-1}m = x-\mu\\
\frac{\partial \log}{\partial S}(x) &= -\frac{1}{2}(xx^\top - S^{-1}mm^\top S^{-1} - S^{-1}) = -\frac{1}{2}(xx^\top - \mu\mu^\top - \Sigma)
\end{align*}
$$


We now implement functions to compute the gradient descent update, using the equations above:

In [106]:
def partial_lq_m(x, p):
    """
    Computes the partial derivative of the log-likelihood function with respect to the mean.

    Args:
    - x (np.array): point at which to evaluate the partial derivative
    - p (NormalRV): distribution

    Returns:
    - partial (np.array): partial derivative of the log-likelihood function with respect to the mean
    """

    partial = x - np.reshape(p.mu, (len(p.mu), 1))
    return partial

def partial_lq_S(x, p):
    """
    Computes the partial derivative of the log-likelihood function with respect to the covariance matrix.

    Args:
    - x (np.array): point at which to evaluate the partial derivative
    - p (NormalRV): distribution

    Returns:
    - res (np.array): partial derivative of the log-likelihood function with respect to the covariance matrix
    """

    x = np.asarray(x)
    if x.ndim == 1:
        x = x[:, np.newaxis]
    n = x.shape[1]
    d = len(p.mu)
    res = np.zeros((d,d,n))
    for i in range(n):
        xi = x[:,i]
        res[:,:,i] = -0.5*(np.outer(xi, xi) - np.outer(p.mu, p.mu)- p.Sigma)
    return res


In [107]:
def is_pd(M):
    """
    Checks if matrix is in the positive definite cone using the Cholesky decomposition.

    Args:
    - M (np.matrix): matrix to check
    """

    try:
        _ = np.linalg.cholesky(M)
        return True
    except np.linalg.LinAlgError:
        return False

In [108]:
def project_pd(M, eps=1e-6):
    """
    Projects a matrix to the positive definite cone. If the matrix is already in the positive definite cone, 
    it is returned unchanged.

    Args:
    - M (np.matrix): matrix to project
    - eps (float): tolerance to set negative and zero eigenvalues to

    Returns:
    - M (np.matrix): projected matrix
    """
    # project a matrix to the positive definite cone
    # M: a symmetric matrix
    # return: a symmetric matrix
    if is_pd(M):
        return M

    # eigen decomposition
    eig_vals, eig_vecs = np.linalg.eig(M)
    # set negative eigenvalues to eps
    eig_vals[eig_vals <= 0] = eps
    # reconstruct matrix
    return eig_vecs @ np.diag(eig_vals) @ eig_vecs.T


In [162]:
def OAIS(phi, pi, q0, lr, nsamples, niter):
    results = []
    for _ in range(niter):
        # compute inner product
        samples = q0.sample(nsamples)
        w = pi.pdf(samples) / q0.pdf(samples) # compute w as we have access to pi
        w2 = w**2
        phi_samples = np.apply_along_axis(phi, 0, samples)
        integral = np.mean(w*phi_samples)/np.mean(w)

        # update q0
        results.append(integral)
        partial_m = partial_lq_m(samples, q0)
        update_m = -np.mean(w2 * partial_m, axis=1)
        
        partial_S = partial_lq_S(samples, q0)
        update_S = -np.mean(w2*partial_S, axis=2)
        
        new_S = project_pd(q0.S - lr*update_S)
        new_m = q0.m - lr*update_m
        q0.update_vals(new_m, new_S)

    return results


In [164]:
        
q = NormalRV(np.array([1,1]), np.array([[5,1],[1,5]]))
pi = NormalRV(np.array([0,0]), np.array([[1,0],[0,1]]))

def phi(x):
    # indicator function for square [-0.5, 0.5]^2
    if np.abs(x[0]) < 0.5 and np.abs(x[1]) < 0.5:
        return 1
    else:
        return 0
e = OAIS(phi, pi, q, 0.01, 1000, 100)

v = sp.stats.norm.cdf(0.5) - sp.stats.norm.cdf(-0.5)
np.abs(e - v**2)


array([5.94752801e-03, 1.38980576e-02, 6.47567485e-03, 5.14608602e-02,
       2.86301991e-03, 1.64091639e-02, 1.16757887e-02, 4.44280026e-03,
       4.91396452e-04, 2.65784535e-03, 1.81488986e-02, 1.21351200e-02,
       1.88690743e-02, 2.54829631e-02, 2.42841599e-02, 1.46844852e-02,
       9.56578589e-03, 1.40737871e-02, 1.18417626e-02, 7.94049891e-03,
       6.37128793e-03, 2.92701639e-04, 5.67596910e-03, 1.52813023e-02,
       6.40132854e-03, 1.41552491e-02, 6.57280330e-03, 1.65068072e-02,
       6.80070918e-03, 7.92005249e-03, 4.18897061e-03, 9.59546862e-03,
       1.78900450e-02, 2.68980815e-02, 9.34662077e-03, 1.56066261e-02,
       2.60442718e-03, 6.40132689e-03, 1.24280506e-03, 1.18878049e-02,
       2.63516927e-03, 6.42078106e-03, 1.12593547e-02, 1.11638251e-02,
       9.80686276e-03, 4.40609320e-03, 2.08479638e-03, 1.36210738e-02,
       1.68901981e-02, 2.11126417e-02, 1.98014011e-02, 7.65251513e-03,
       4.57952585e-03, 8.52280197e-03, 1.82682619e-02, 9.27244299e-03,
      